# LSTM을 활용한 불쾌지수 예측

In [22]:
#!pip install tensorflow==1.15

In [9]:
#conda install python=3.6.10


Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
print(tf.__version__)

2.9.1


- 랜덤에 의해 똑같은 결과를 재현하도록 시드 설정
- 하이퍼파라미터를 튜닝하기 위한 용도
- Neural net의 초기값을 고정

In [2]:
tf.set_random_seed(608)

In [3]:
# Standardization
def data_standardization(x):
    x_np = np.asarray(x)
    return (x_np - x_np.mean()) / x_np.std()

- 너무 작거나 너무 큰 값이 학습을 방해하는 것을 방지하고자 정규화한다
- x가 양수라는 가정하에 최소값과 최대값을 이용하여 0~1사이의 값으로 변환
- Min-Max scaling

In [4]:
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7)
    # 1e-7은 0으로 나누는 오류 예방차원

- 정규화된 값을 원래의 값으로 되돌린다
- 정규화하기 이전의 org_x값과 되돌리고 싶은 x를 입력하면 역정규화된 값을 리턴한다

In [5]:
def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()

In [6]:
# 하이퍼파라미터
input_data_column_cnt = 3  # 입력데이터의 컬럼 개수(Variable 개수)
output_data_column_cnt = 1  # 결과데이터의 컬럼 개수

In [7]:
seq_length = 30  # 1개 시퀀스의 길이(시계열데이터 입력 개수)
rnn_cell_hidden_dim = 20  # 각 셀의 (hidden)출력 크기
forget_bias = 1.0  # 망각편향(기본값 1.0)
num_stacked_layers = 1  # stacked LSTM layers 개수
keep_prob = 1.0  # dropout할 때 keep할 비율

epoch_num = 1000  # epoch 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01  # 학습률

- 데이터 로딩

In [8]:
stock_file_name = 'DiscomfortIndex.csv'  # 부산광역시 기온 자료
encoding = 'euc-kr'  # 문자 인코딩 방식 설정
raw_dataframe = pd.read_csv(stock_file_name, encoding=encoding)  # 판다스이용 csv파일 로딩
raw_dataframe.info()  # 데이터 정보 출력

temp_info = raw_dataframe.values[1:].astype(np.float)  # 기온 자료를 부동소수점형으로 변환
print("temp_info.shape: ", temp_info.shape)
print("temp_info[0]: ", temp_info[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 49 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1970년   32 non-null     float64
 1   1971년   32 non-null     float64
 2   1972년   32 non-null     float64
 3   1973년   32 non-null     float64
 4   1974년   32 non-null     float64
 5   1975년   32 non-null     float64
 6   1976년   32 non-null     float64
 7   1977년   32 non-null     float64
 8   1978년   32 non-null     float64
 9   1979년   32 non-null     float64
 10  1980년   32 non-null     float64
 11  1981년   32 non-null     float64
 12  1982년   32 non-null     float64
 13  1983년   32 non-null     float64
 14  1984년   32 non-null     float64
 15  1985년   32 non-null     float64
 16  1986년   32 non-null     float64
 17  1987년   32 non-null     float64
 18  1988년   32 non-null     float64
 19  1989년   32 non-null     float64
 20  1990년   32 non-null     float64
 21  1991년   32 non-null     float64
 22  1992

C:\Users\admin\AppData\Local\Temp\ipykernel_17224\1616782163.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  temp_info = raw_dataframe.values[1:].astype(np.float)  # 기온 자료를 부동소수점형으로 변환


In [9]:
# 데이터 전처리
# 데이터 정규화

norm_temp = min_max_scaling(temp_info)  # 기온 데이터 정규화 처리

In [10]:
# 행은 그대로 두고 열을 우측에 붙여 합친다
x = norm_temp
print("x.shape: ", x.shape)
print("x[0]: ", x[0])  # x의 첫 값
print("x[-1]: ", x[-1])  # x의 마지막 값
print("=" * 100)  # 화면상 구분용

y = x[:, [-1]]  # 타켓은 평균기온이다
print("y[0]: ", y[0])  # y의 첫 값
print("y[-1]: ", y[-1])  # y의 마지막 값

dataX = []  # 입력으로 사용될 Sequence Data
dataY = []  # 출력(타켓)으로 사용

for i in range(0, len(y) - seq_length):
    _x = x[i: i + seq_length]
    _y = y[i + seq_length]  # 다음날의 평균기온(정답)
    if i is 0:
        print(_x, "->", _y)  # 첫번째 행만 출력해 봄
    dataX.append(_x)  # dataX 리스트에 추가
    dataY.append(_y)  # dataY 리스트에 추가

x.shape:  (31, 49)
x[0]:  [0.32041762 0.32041762 0.57693175 0.51736842 0.70375908 0.69211021
 0.52005677 0.6572152  0.78452327 0.46824625 0.43042691 0.72767807
 0.45814667 0.82604038 0.67020254 0.62701617 0.7689624  0.64563951
 0.70427791 0.45369112 0.84051661 0.3867891  0.65990564 0.42151176
 0.84655313 0.75296827 0.85825079 0.87879803 0.6722467  0.26510019
 0.71621976 0.79375209 0.62611927 0.57058455 0.69215613 0.66111199
 0.71845114 0.77465769 0.67581999 0.59488117 0.7222542  0.63152774
 0.80368545 0.74922721 0.85418763 0.70161551 0.68514866 0.8885056
 1.        ]
x[-1]:  [0.33186348 0.33186348 0.42451914 0.68012127 0.56924337 0.67211434
 0.46585512 0.45990684 0.62492385 0.49595329 0.37099234 0.50020356
 0.59263824 0.61107957 0.65712002 0.66225255 0.51277969 0.53620526
 0.6217383  0.55470174 0.65336836 0.54938374 0.52642151 0.42107588
 0.73445829 0.66332956 0.59996255 0.58495457 0.55179786 0.6049652
 0.64767342 0.61231055 0.49201977 0.47546896 0.59172079 0.54412799
 0.64976806 0.646

<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\admin\AppData\Local\Temp\ipykernel_17224\2397506268.py:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if i is 0:


In [11]:
# 학습용/테스트용 데이터 생성
# 전체 70%를 학습용 데이터로 사용
train_size = int(len(dataY) * 0.7)
# 나머지(30%)를 테스트용 데이터로 사용
test_size = len(dataY) - train_size

In [12]:
# 데이터를 잘라 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])

# 데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

In [13]:
# 텐서플로우 플레이스홀더 생성
# 입력 X, 출력 Y를 생성한다

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

X = tf.placeholder(shape=[None, seq_length, input_data_column_cnt], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

print("X : ", X)
print("Y : ", Y)

X :  Tensor("Placeholder:0", shape=(?, 30, 3), dtype=float32)
Y :  Tensor("Placeholder_1:0", shape=(?, 1), dtype=float32)


In [14]:
# 검증용 측정지표를 산출하기 위한 targets, predictions를 생성한다

targets = tf.placeholder(shape=[None, 1], dtype = tf.float32)
predictions = tf.placeholder(shape=[None, 1], dtype=tf.float32)

print("targets: ", targets)
print("predictions: ", predictions)

targets:  Tensor("Placeholder_2:0", shape=(?, 1), dtype=float32)
predictions:  Tensor("Placeholder_3:0", shape=(?, 1), dtype=float32)


In [15]:
# 모델(LSTM 네트워크) 생성
def lstm_cell():
    # LSTM셀을 생성
    # num_units: 각 Cell 출력 크기
    # forget_bias:  to the biases of the forget gate
    #              (default: 1)  in order to reduce the scale of forgetting in the beginning of the training.
    # state_is_tuple: True ==> accepted and returned states are 2-tuples of the c_state and m_state.
    # state_is_tuple: False ==> they are concatenated along the column axis.
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim,
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell

In [19]:
# num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()

AttributeError: module 'tensorflow.compat.v1' has no attribute 'contrib'

In [20]:
# RNN Cell(여기서는 LSTM셀임)들을 연결
hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)

# [:, -1]를 잘 살펴보자. LSTM RNN의 마지막 (hidden)출력만을 사용했다.
# 과거 여러 날짜의 기온 자료를 이용해서 다음날의 평균기온 1개를 예측 : MANY-TO-ONE model
hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], output_data_column_cnt, activation_fn=tf.identity)

NameError: name 'multi_cells' is not defined

In [27]:
# 손실함수 : 평균제곱오차
loss = tf.reduce_sum(tf.square(hypothesis - Y))
# AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.RMSPropOptimizer(learning_rate) # LSTM과 궁합 별로임

train = optimizer.minimize(loss)

# RMSE(Root Mean Square Error)
# 제곱오차의 평균을 구하고 다시 제곱근을 구하면 평균 오차가 나온다
# rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions))) # 아래 코드와 같다
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))

train_error_summary = []  # 학습용 데이터의 오류를 중간 중간 기록한다
test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
test_predict = ''  # 테스트용데이터로 예측한 결과

sess = tf.Session()
sess.run(tf.global_variables_initializer())


NameError: name 'hypothesis' is not defined

In [28]:
# 학습한다
start_time = datetime.datetime.now()  # 시작시간을 기록한다
print('Run : model training')
for epoch in range(epoch_num):
    _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
    if ((epoch + 1) % 100 == 0) or (epoch == epoch_num - 1):  # 100번째마다 또는 마지막 epoch인 경우
        # 학습용데이터로 rmse오차를 구한다
        train_predict = sess.run(hypothesis, feed_dict={X: trainX})
        train_error = sess.run(rmse, feed_dict={targets: trainY, predictions: train_predict})
        train_error_summary.append(train_error)

        # 테스트용데이터로 rmse오차를 구한다
        test_predict = sess.run(hypothesis, feed_dict={X: testX})
        test_error = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
        test_error_summary.append(test_error)

        # 현재 오류를 출력한다
        print("epoch: {}, train_error(A): {}, test_error(B): {}, B-A: {}".format(epoch + 1, train_error, test_error,
                                                                                 test_error - train_error))

end_time = datetime.datetime.now()  # 종료시간을 기록한다
elapsed_time = end_time - start_time  # 경과시간을 구한다
print('elapsed_time:', elapsed_time)
print('elapsed_time per epoch:', elapsed_time / epoch_num)

Run : model training


NameError: name 'sess' is not defined

In [29]:
# 하이퍼파라미터 출력
print('input_data_column_cnt:', input_data_column_cnt, end='')
print(',output_data_column_cnt:', output_data_column_cnt, end='')

print(',seq_length:', seq_length, end='')
print(',rnn_cell_hidden_dim:', rnn_cell_hidden_dim, end='')
print(',forget_bias:', forget_bias, end='')
print(',num_stacked_layers:', num_stacked_layers, end='')
print(',keep_prob:', keep_prob, end='')

print(',epoch_num:', epoch_num, end='')
print(',learning_rate:', learning_rate, end='')

print(',train_error:', train_error_summary[-1], end='')
print(',test_error:', test_error_summary[-1], end='')
print(',min_test_error:', np.min(test_error_summary))

input_data_column_cnt: 3,output_data_column_cnt: 1,seq_length: 30,rnn_cell_hidden_dim: 20,forget_bias: 1.0,num_stacked_layers: 1,keep_prob: 1.0,epoch_num: 1000,learning_rate: 0.01

NameError: name 'train_error_summary' is not defined

In [30]:
# 결과 그래프 출력
plt.figure(1)
plt.plot(train_error_summary, 'gold')
plt.plot(test_error_summary, 'b')
plt.xlabel('Epoch(x100)')
plt.ylabel('Root Mean Square Error')

plt.figure(2)
plt.plot(testY, 'r')
plt.plot(test_predict, 'b')
plt.xlabel('Time Period')
plt.ylabel('AirTemp(Daily Mean)')
plt.show()

# sequence length만큼의 가장 최근 데이터를 슬라이싱한다
recent_data = np.array([x[len(x) - seq_length:]])
print("recent_data.shape:", recent_data.shape)
print("recent_data:", recent_data)

NameError: name 'train_error_summary' is not defined

<Figure size 432x288 with 0 Axes>

In [31]:
# 내일 평균기온을 예측해본다
test_predict = sess.run(hypothesis, feed_dict={X: recent_data})

print("test_predict", test_predict[0])
test_predict = reverse_min_max_scaling(temp_info, test_predict)  # 기온값을 역정규화
print("Tomorrow's Daily Mean Air Temperature", test_predict[0])  # 예측한 평균기온

NameError: name 'sess' is not defined